In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.training import MuModel
from src.training.losses import Loss

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Let's suppose the cartpole env, which has a 4-dim observation space and a 2-dim action space

K = 3
N = 5
action_dim = 2
observation_dim = 4

lr = nn.MSELoss()
lv = nn.MSELoss()
lp = nn.CrossEntropyLoss()

criterion = Loss(lr=lr,lv=lv,lp=lp)

model = MuModel(
    observation_dim=observation_dim,
    action_dim=action_dim,
    state_dim=128,
    N=5,
    K=3,
    criterion=criterion
)

# some random inputs
batch_size = 1

horizon_target = torch.randint(1,K+1,size=(batch_size,))
# horizon_target = torch.tensor([3])

batch = (
    torch.rand(batch_size, N, observation_dim), # observations
    torch.rand(batch_size, K, action_dim),      # target policies
    torch.rand(batch_size, K, action_dim),      # target actions
    torch.rand(batch_size, K),                  # target rewards
    torch.rand(batch_size, K),                  # target returns
    horizon_target
)

print(horizon_target)

tensor([3])


/Users/guillaumeberthelot/anaconda3/envs/SDD_Torch/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
# forward pass
model.training_step(batch)

/Users/guillaumeberthelot/anaconda3/envs/SDD_Torch/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(4.8459, grad_fn=<AddBackward0>)

In [5]:
model.validation_step(batch)

4.684188365936279

In [6]:
n_step_grad = 5
n_epoch = 5

for k in range(n_epoch):
    print(f"Epoch {k}")
    for _ in range(n_step_grad):
        model.training_step(batch)
    model.validation_step(batch)


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
